# Machine Learning & Application 
## 2021-2022
# Projet Rubik's cube

Le projet porte sur un problème d'apprentissage supervisé. Le problème fait parti des données du [challenge des données](https://challengedata.ens.fr/challenges/20) ENS et s'intitule "Solve 2x2x2 Rubik's cube" et est présenté par la société LumenAI. Une [vidéo](https://www.college-de-france.fr/video/stephane-mallat/2019) décrivant le problème se trouve sur le site du collège de France. 

Les autres challenges sont aussi très intéressants, mais nécessitent plus de connaissances en machine learning (par exemple de l'apprentissage sur des séries temporelles, sur des images, des sons, du texte, etc...). D'où le choix de ce challenge dont les données sont très proches de problèmes sur lesquels on a travaillé.

La résolution d'un rubik's cube peut être vu comme un problème d'intelligence artificielle (par exemple en utilisant des techniques de recherches avec des heuristiques). On peut même étudier le graphe du jeu du point de vue de la théorie des graphes et découvrir qu'en fait il existe toujours un chemin relativement court à une solution.

ex dans la litérature:
 * "The Diameter of the Rubik's Cube Group Is Twenty", *T. Rokicki, H. Kociemba, M. Davidson, and J. Dethridge*, SIAM Review, 2014, Vol. 56, No. 4 : pp. 645-670.
 * "Solving Rubik’s Cube Using Graph Theory", Khemani C., Doshi J., Duseja J., Shah K., Udmale S., Sambhe V. (2019) in: Verma N., Ghosh A. (eds) Computational Intelligence: Theories, Applications and Future Directions - Volume I. Advances in Intelligent Systems and Computing, vol 798. Springer, Singapore. 
 
Ici, on a une base de données qui contient la description de rubik's cubes ainsi que le nombre minimal de coups pour les résoudre. On ne sait pas comment les problèmes ont été générés (est-ce que cela entraine un biais dans les problèmes, est-ce que plusieurs problèmes similaires sont présents dans la base? (Ici par similaire, on pourrait peut être avoir deux problèmes qui apparaissent dans la base, mais en permuttant certaines couleurs, on aurait peut-être exactement le même problème!)). Cependant, on vous demande de constuire un modèle pour nous aider à prédire le nombre de coups minimal pour un problème donné. Ensuite, vous pourriez utiliser ce modèle dans un algorithme de recherche étudié en cours d'IA.

On peut le voir comme un problème de regression où il faut deviner le nombre minimal de coups pour résoudre le rubik's cube, ou comme un problème de classification où la classe d'un état du rubik's cube est le nombre minimal de coups pour le résoudre (donc on pourrait avoir au plus 19 classes). Toutes les méthodes que l'on a vu en cours peuvent s'appliquer.

## Les données et le site du challenge

Le projet s'effectue en binôme. Vous devez ouvrir un compte pour le binôme sur le site du challenge, choisissez de participer seul (le binôme sera un seul participant au challenge), puis inscrivez-vous au challenge du cours *M1 MIAGE Dauphine - PSL - 2021-2022*.

Vous aurez accès à trois ensembles: 
 * `x_train` qui contient la description de 1.837.079 différents rubik's cubes. Chaque rubik's cube est représenté par 25 attributs (lisez la description sur le site du challenge).
 * `y_train` qui contient le nombre minimal de coups pour résoudre chacun des 1.837.079 différents rubik's cubes. 
 Ces données sont vos données d'entrainement.
 * enfin `x_test` qui contient la description de 1.837.080 nouveaux rubik's cubes. Vous ne connaissez pas le nombre minimal pour chacun de ces problèmes. 
 
Pour participer aux challenges, il vous faudra uploader sur le site votre prédiction sur les rubik's cubes du fichier `x_test` et le site du challenge vous donnera un score. Pour ce score, le site utilise l'erreur moyenne absolue: pour les n=1.837.080 exemples du fichier test, on fait la moyenne entre le vrai nombre minimal de coups $y_i$ et votre prédiction $z_i$: 
$$ \frac{\sum_{i=1}^n |y_i-z_i|}{n}$$

Malheureusement (pour vous), le site ne vous donnera pas plus d'information que votre score, vous ne pourrez pas savoir quelles sont vos bonnes prédictions et quelles sont vos erreurs. Pire, le site vous permettra d'uploader une prédiction que deux fois par jour!

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


dfX = pd.read_csv("train_input.csv")
X = dfX.to_numpy()
X = X[:,1:]
dfY = pd.read_csv("train_output.csv")
y = dfY.to_numpy()
y = y[:,1:]
print("taille des données:")
print("X:", X.shape)
print("y:", y.shape)

dfT = pd.read_csv("test_input.csv")
Xtest = dfT.to_numpy()
print("test:", Xtest.shape)

taille des données:
X: (1837079, 24)
y: (1837079, 1)
test: (1837080, 25)


In [8]:

print("Il y a ", X.size, " observations avec ", X[0].size, " attributs")

Il y a  44089896  observations avec  24  attributs


In [9]:
dfX = dfX.iloc[:,1:]

In [10]:
dfY = dfY.iloc[:,1:]


In [11]:
dfX

,pos0,pos1,pos2,pos3,pos4,pos5,pos6,pos7,pos8,pos9,...,pos14,pos15,pos16,pos17,pos18,pos19,pos20,pos21,pos22,pos23
0,4,1,1,1,6,2,6,6,5,4,...,4,4,1,3,3,5,2,3,5,3
1,6,5,2,1,2,2,6,3,4,4,...,4,1,3,1,5,3,5,3,5,2
2,5,3,3,2,3,1,6,5,1,1,...,4,6,4,4,4,3,2,2,5,2
3,5,5,4,1,2,1,6,1,2,2,...,4,4,1,6,6,3,6,3,5,5
4,4,2,1,5,1,3,6,6,3,3,...,4,2,6,6,2,1,2,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837074,2,1,3,3,5,3,6,6,1,5,...,4,3,5,4,2,6,6,1,5,2
1837075,2,3,3,5,6,4,6,1,3,1,...,4,5,1,4,6,1,2,6,5,2
1837076,3,3,3,2,2,4,6,6,1,4,...,4,3,2,6,1,5,1,1,5,2
1837077,5,3,5,1,5,3,6,3,6,4,...,4,4,2,6,4,2,2,6,5,1


In [12]:
dfY

,distance
0,11
1,11
2,11
3,9
4,12
...,...
1837074,11
1837075,9
1837076,12
1837077,11


In [13]:
dfY.value_counts()

distance
11          675426
10          465294
12          391268
9           180254
8            57074
13           45140
7            16529
6             4485
5             1128
4              267
14             138
3               60
2               13
1                3
dtype: int64

In [29]:
from sklearn.tree import DecisionTreeClassifier
from math import sqrt
from sklearn.metrics import precision_score
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
dtree = DecisionTreeClassifier(criterion="entropy", max_depth=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)
dtree.fit(X_train, y_train)
ydt = dtree.predict(X_test)
somme = 0
print(np.size(ydt),np.size(y_test))
print(mean_absolute_error(y_test,ydt))

183708 183708
0.857469462407734
